In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('/home/groups/ZuckermanLab/jalim/instalLocal/celltraj/celltraj')
import trajCellPoseSr
import h5py
import pickle
import os
import subprocess
import time
import string
import itertools
import pandas
import re
import scipy
from datetime import date

In [2]:
trajl = 40
nstates_init = 7
today = date.today()
date2day = today.strftime("%b%d-%Y")
figid = 'LI204601_P_tlen'+str(trajl)+'_'+date2day+'_nS'+str(nstates_init) 
datapath = '/home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/bulkRNAseq/scripts'
seqFile0 = 'MDDligandCombRNAseqLog2TPM_proteinCoding.csv'
seqData0 = pandas.read_csv(datapath+'/'+seqFile0)

In [3]:
# Create a filter for log2(TPM) > 0.5 
ind_minexpr = np.where(np.sum(seqData0.iloc[:, 3:] > 0.5, axis=1) >= 3)[0]
geneNames0 = seqData0['hgnc_symbol']
ind_nan = np.where(np.logical_not(pandas.isna(seqData0['hgnc_symbol'])))[0] # also genes with names
ensembl_gene_id0 = seqData0['ensembl_gene_id']
# Indices of genes that are expressed (excluding NaN value members)
ind_expressed = np.intersect1d(ind_minexpr, ind_nan) 
gene_names = geneNames0[ind_expressed] # Genes that are expressed
ensembl_gene_ids = ensembl_gene_id0[ind_expressed]

In [4]:
# read in Deseq2 files
cond0 = ['OSM','EGF','EGFTGFB','TGFB','PBS','OSMTGFBEGF','OSMEGF','OSMTGFB']
ncond0 = len(cond0)
deseq0 = [None]*ncond0
for icond in range(ncond0):
    seqfile = f'analysis_R/deseq2_DE_lfcshrink_ligands_{cond0[icond]}_vs_CTRL.csv'
    deseq0[icond] = pandas.read_csv(datapath+'/'+seqfile)

In [5]:
# now match the genes in the two datasets together, we will do into protein coding nGenes0
nGenes0 = ind_expressed.size
inds_dataset0 = np.zeros(ncond0).astype(int)
#inds_dataset1 = np.ones(ncond1).astype(int)
#inds_dataset = np.append(inds_dataset0,inds_dataset1)
inds_dataset = inds_dataset0
inds0 = np.where(inds_dataset == 0)[0]
#inds1 = np.where(inds_dataset == 1)[0]
nsamples = inds_dataset.size
x_lfc = np.ones((nsamples, nGenes0))*np.nan # logarithmic fold change
x_padj = np.ones((nsamples, nGenes0))*np.nan
seq_genes0 = deseq0[0]['Unnamed: 0']
#seq_genes1 = deseq1[0]['Unnamed: 0']

In [6]:
for i in range(nGenes0):
    if i%100 == 0:
        print(f'matching gene {str(i)} of {str(nGenes0)}')
    gene_name = ensembl_gene_ids.iloc[i]
    indgene1 = np.where(seq_genes0 == gene_name)[0] 
    if indgene1.size > 0:
        for icond in range(ncond0):
            lfc = deseq0[icond].iloc[ind_expressed[i]]['log2FoldChange']
            padj = deseq0[icond].iloc[ind_expressed[i]]['padj']
            x_lfc[inds0[icond], i] = lfc
            x_padj[inds0[icond], i] = padj

matching gene 0 of 14577
matching gene 100 of 14577
matching gene 200 of 14577
matching gene 300 of 14577
matching gene 400 of 14577
matching gene 500 of 14577
matching gene 600 of 14577
matching gene 700 of 14577
matching gene 800 of 14577
matching gene 900 of 14577
matching gene 1000 of 14577
matching gene 1100 of 14577
matching gene 1200 of 14577
matching gene 1300 of 14577
matching gene 1400 of 14577
matching gene 1500 of 14577
matching gene 1600 of 14577
matching gene 1700 of 14577
matching gene 1800 of 14577
matching gene 1900 of 14577
matching gene 2000 of 14577
matching gene 2100 of 14577
matching gene 2200 of 14577
matching gene 2300 of 14577
matching gene 2400 of 14577
matching gene 2500 of 14577
matching gene 2600 of 14577
matching gene 2700 of 14577
matching gene 2800 of 14577
matching gene 2900 of 14577
matching gene 3000 of 14577
matching gene 3100 of 14577
matching gene 3200 of 14577
matching gene 3300 of 14577
matching gene 3400 of 14577
matching gene 3500 of 14577
matc

In [14]:
tmSet = cond0 
nLigConds = len(tmSet)
sctm = trajCellPoseSr.cellPoseTraj()

inds_finite = np.where(np.isfinite(np.sum(x_lfc, axis=0)))[0]
x_lfc = x_lfc[:, inds_finite]
x_padj = x_padj[:, inds_finite]
gene_names = gene_names.iloc[inds_finite]
ensembl_gene_ids = ensembl_gene_ids.iloc[inds_finite]
Xpca, pca = sctm.get_pca_fromdata(x_lfc, var_cutoff = .95)
colorSet = ['gray', 'gold', 'red', 'blue', 'orange', 'green', 'purple', 'brown',
            'gray', 'gold', 'lightblue', 'red', 'lightgreen', 'darkred', 'green']

In [23]:
# get morphodynamical state probabilities from imaging analysis: To Change
stProbFile = datapath+'/'+'stateprobs_LI204601_P_tlen'+str(trajl)+'_Aug18-2023_nS'+str(nstates_init)+'.dat' 

if not stProbFile:
  print("ERROR in reading state probability file")
  sys.exit(0)

state_probs = np.loadtxt(stProbFile)
tmSet_imaging = np.array(['OSM','EGF','EGFTGFB','TGFB','PBS','OSMTGFBEGF','OSMEGF','OSMTGFB',
                          'PBS1','EGF1','OSM1','TGFB1','OSMEGF1'])
tmfSet = tmSet #so much fun with names
inds_tmfSet_imaging = np.array([]).astype(int)
for imf in range(len(tmfSet)):
    tm = tmfSet[imf]
    inds_tmfSet_imaging = np.append(inds_tmfSet_imaging, np.where(tmSet_imaging == tm)[0])

inds_tmfSet_Imaging = inds_tmfSet_imaging
state_probs_ = state_probs_[inds_tmfSet_imaging, :]
print("List of all conditions:", np.array(tmSet)[inds_tmfSet_imaging]) # Test + training sets

List of all conditions: ['OSM' 'EGF' 'EGFTGFB' 'TGFB' 'PBS' 'OSMTGFBEGF' 'OSMEGF' 'OSMTGFB']


In [24]:
# Predict fold-change value of genes of the "test set"
def get_predictedFC(state_probs_test, statesFC):
    n_test = state_probs_test.shape[0]
    nStates = state_probs_test.shape[1]
    nGenes = statesFC.shape[1]
    x_FC_predicted = np.ones((n_test, nGenes))*np.nan
    for ig in range(nGenes):
        v_states = statesFC[:, ig]
        if np.sum(np.logical_or(np.isnan(v_states), np.isinf(v_states))) == 0: # if v_states is neither NaN nor INF
            for it in range(n_test):
                x_FC_predicted[it, ig] = np.sum(np.multiply(state_probs_test[it, :], statesFC[:, ig]))
    return x_FC_predicted

In [25]:
def get_state_decomposition(x_fc, state_probs, ncombinations=500, inds_tm_training=None,
                            save_file=None, visual=False, verbose=True, nchunk=100, gene_names=None):
    nStates = state_probs.shape[1]
    ntr = state_probs.shape[0]
    nGenes = x_fc.shape[1]
    ntr_measured = x_fc.shape[0]
    if nStates > ntr:
        print(f'error, more states than conditions in state probabilities')
        return
    if nStates > ntr_measured:
        print(f'error, more states than measured bulk conditions')
        return
    x_fc_states = np.ones((nStates, nGenes))*np.nan
    if inds_tm_training is None:
        inds_tm_training = np.arange(ntr).astype(int)
    ntr_training = inds_tm_training.size
    comb_trainarray = np.array(list(itertools.combinations(inds_tm_training, nStates)))
    ncomb = comb_trainarray.shape[0]
    print(f'{ncomb} possible combinations of {ntr} training measurements decomposed into {nStates} states')
    if ncombinations > ncomb:
        ncombinations = ncomb
    print(f'using {ncombinations} of {ncomb} possible training set combinations randomly per feature')
    for ig in range(nGenes): # LOOP OVER NUMBER OF GENES
        # Generate a uniform random sample from np.arange(ncomb) of size ncombinations
        indr = np.random.choice(ncomb, ncombinations, replace=False)
        if ig%nchunk == 0 and verbose:
            print(f'decomposing gene {ig} of {nGenes}')
            if save_file is not None:
                np.save(save_file, x_fc_states)
        v_states_comb = np.zeros((ncombinations, nStates))
        for icomb in range(ncombinations):
            indcomb = comb_trainarray[indr[icomb]] # Get randomized index from above 
            v_treatments = x_fc[indcomb, ig]
            # Least square linear optimization for each Gene --> solving state_probs*x = v_treatments (fold-change)  
            res = scipy.optimize.lsq_linear(state_probs[indcomb, :], v_treatments, bounds=(lb, ub), verbose=1)
            v_states_comb[icomb, :] = res.x.copy() # x is returned from scipy.optimize.lsq_linear 
        v_states = np.mean(v_states_comb, axis=0)
        x_fc_states[:, ig] = v_states.copy()
        if ig%nchunk == 0 and visual:
            plt.clf()
            plt.plot(v_states_comb.T, 'k.')
            plt.plot(v_states.T, 'b-', linewidth=2)
            if gene_names is None:
                plt.title(f'{ig} of {nGenes}')
            else:
                plt.title(str(gene_names.iloc[ig])+' gene '+str(ig)+' of '+str(nGenes))
            plt.pause(.1)
    if save_file is not None:
        np.save(save_file, x_fc_states)
    return x_fc_states

In [1]:
visual = False
plt.clf()
plt.figure(figsize = (9, 6))
ax = plt.gca()
inds_tm_All = np.arange(nLigConds).astype(int)

############### MODIFY INDICES AND DATA ACCORDING TO WHETHER A CONDITION IS EXCLUDED FROM TRAINING ###############
loco = True
for iTest in range(nLigConds):
    
    # leaving one "LIGAND" condition out (LOCO), just test from combo data
    inds_tm_test = np.array([iTest]).astype(int) 
    LOCO = tmSet_imaging[inds_tm_test] # Get condition's name
    LOCO = ''.join(LOCO) # convert string list to string
    # remove LOCO index from All conditions => only a "training set"
    inds_tm_training = np.setdiff1d(inds_tm_All, inds_tm_test) 
    ############# Update state probabilities and log-fold change values as per "inds_tm_training" #############
    state_probs_training = state_probs[inds_tm_training, :]
    x_lfc_training = x_lfc[inds_tm_training, :]
    inds_tmfSet_imaging = np.arange(len(inds_tm_training), dtype = int)
    inds_tm_training = inds_tmfSet_imaging # Update training indices after LOCO
    nStates = state_probs_training.shape[1] # Number of Macroscopic (morphodynamic) states 
    ntr = state_probs_training.shape[0] # Number of training conditions 
    seq_genes = gene_names.reset_index(drop = True)
    ntr_training = inds_tm_training.size # Number of conditions to perform training 
    lb = np.zeros(nStates)
    ub = np.ones(nStates)*np.inf
     
    # First pass, just with training set
    nGenes = x_lfc.shape[1]
    # Element-wise raise 2 to the power of x_lfc --> Eliminate Log @ base 2
    x_fc_training = 2**x_lfc_training # Log-fold change values of the training set only
    x_fc_all = 2**x_lfc # Log-fold change values of test & training sets
    
    state_names = np.array(list(string.ascii_uppercase))[0:nStates]
    get_counts = True
    ################## Decomposition of Genes into the given "nStates" ###################
    if get_counts:
        x_fc_states = get_state_decomposition(x_fc_training, state_probs_training, ncombinations=500, 
                                              inds_tm_training=inds_tm_training, 
                                              save_file='statefc_production_'+figid+'_'+LOCO+'.npy', 
                                              visual=visual, gene_names=gene_names)
    else:
        x_fc_states = np.load('statefc_production_'+figid+LOCO+'.npy')
             
    state_probs_test = state_probs_[inds_tm_test, 0:nStates] # State probabilities of the "Test Set"
    # Predict fold-change values of the test set whereas the model was trained on the training set
    x_fc_predicted = get_predictedFC(state_probs_test, x_fc_states)
    x_lfc_predicted = np.log2(x_fc_predicted) # Convert from fold-change to log fold-change
    
    print(f'{tmSet[iTest]} corr: {np.corrcoef(x_lfc[iTest, :], x_lfc_predicted[0, :])[0, 1]:.2f}')

    nConds_test = len(inds_tm_test) # Number of Ligand conditions in "Test Set"
     
    ######################### how unique are state probabilities #########################
    get_correlations = True
    if get_correlations:
        nrandom = 500
        corrSet_pred = np.zeros(nConds_test) # Correlation between predicted and real values
        corrSet_predrand = np.zeros((nrandom, nConds_test)) # Correlation between NULL model and predicted values
        corrSet_rand = np.zeros((nrandom, nConds_test)) # Correlation of NULL model and real values
        for ir in range(nrandom):
            state_probs_r = np.zeros_like(state_probs_test) # state probabilities random -> NULL model
            for it in range(nConds_test):
                rp = np.random.rand(nStates) # Random probability of each training set  
                rp = rp/np.sum(rp)
                state_probs_r[it, :] = rp.copy()
            x_fc_null = get_predictedFC(state_probs_r, x_fc_states)
            for it in range(nConds_test):
                lfc_pred = np.log2(x_fc_predicted[it, :])
                lfc_real = x_lfc[iTest, :] # "original" Log-fold change value(s) of test condition(s)
                #.5*x_counts_all[indcombos[i,0],:]+.5*x_counts_all[indcombos[i,1],:]
                lfc_null = np.log2(x_fc_null[it, :]) 
                df = pandas.DataFrame(np.array([lfc_pred, lfc_null, lfc_real]).T)
                rhoSet = df.corr().to_numpy() # Convert pandas data type to Numpy array 
                corrSet_pred[it] = rhoSet[0, 2]
                corrSet_rand[ir, it] = rhoSet[1, 2]
                corrSet_predrand[ir, it] = rhoSet[0, 1]
                print(tmfSet[iTest], "correlations: prediction", "%.2f" % rhoSet[0, 2],", null",
                      "%.2f" % rhoSet[1, 2],", prednull", "%.2f" % rhoSet[0, 1],", indices random:", ir)
                if ir%100 == 0 or ir == nrandom - 1:
                    np.save('corrSet_'+figid+'_'+LOCO+'_production.npy', corrSet_rand)
                    np.save('corrSet_predrand_'+figid+'_'+LOCO+'_production.npy', corrSet_predrand)
                    np.save('corrSet_pred_'+figid+'_'+LOCO+'_production.npy', corrSet_pred)
    else:
        corrSet_rand = np.load('corrSet_'+figid+'_'+LOCO+'_production.npy')
        corrSet_predrand = np.load('corrSet_predrand_'+figid+'_'+LOCO+'_production.npy')
        corrSet_pred = np.load('corrSet_pred_'+figid+'_'+LOCO+'_production.npy')

    ################################# Plot model predictions for LOCO  ################################
    it = 0 # Index = 0 in case of LOCO
    vplot = ax.violinplot(corrSet_rand[:, it], positions = [iTest + 1],
                          showmeans = True, showextrema = False, quantiles = [.025, .975])
    for partname in ('cmeans', 'cquantiles'):
        vp = vplot[partname]
        vp.set_edgecolor('black')
    plt.scatter(iTest + 1, corrSet_pred[it], s=100, c = 'red', marker = 'd')
    for pc in vplot['bodies']:
        pc.set_facecolor('black')
        pc.set_edgecolor('black')
        # pc.set_alpha(1)
    plt.pause(.1)
    
ax.set_xticks(np.arange(1, len(inds_tmfSet_Imaging) + 1))
ax.set_xticklabels(np.array(tmfSet)[inds_tmfSet_Imaging])
plt.setp(ax.get_xticklabels(), rotation = 30, ha = "right", rotation_mode = "anchor")
plt.ylabel('correlation'); plt.ylim(-1., 1.)
plt.savefig(figid+'_LOCO_correlations.png')
plt.close()

NameError: name 'plt' is not defined